In [1]:
# pip install -q "transformers==4.9.2"

zsh:1: command not found: pip


In [ ]:
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

In [ ]:
#Model
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

#Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [ ]:
def bert_model(question, context):
    encoding = tokenizer.encode_plus(text=question,text_pair=context)
    inputs = encoding['input_ids']  #Token embeddings
    sentence_embedding = encoding['token_type_ids']  #Segment embeddings
    tokens = tokenizer.convert_ids_to_tokens(inputs) #input tokens

    output = model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))
    start_index = torch.argmax(output.start_logits)

    end_index = torch.argmax(output.end_logits)

    answer = ' '.join(tokens[start_index:end_index+1])

    corrected_answer = ''

    for word in answer.split():
    
        #If it's a subword token
        if word[0:2] == '##':
            corrected_answer += word[2:]
        else:
            corrected_answer += ' ' + word

    return corrected_answer

In [2]:
cont_file = "output/resume_content.txt"

In [4]:
from ast import literal_eval
out = open("output/education.txt", 'w+')
with open(cont_file, 'r') as f:
    for line in f:
        content = list(literal_eval(line))
        for sec in content:
            title = sec['title']
            id = sec['id']
            if title == "Education":
                context = ' '.join(sec['content'])
                ques_college = "what is your college?"
                ques_major = "what is your major?"
                college = bert_model(ques_college, context)
                major = bert_model(ques_major, context)
                out.write('{}, {}, {}, {}\n'.format(id, college, major,context))

{'id': '16852973', 'title': 'Summary', 'content': ['Dedicated Customer Service Manager with 15+ years of experience in Hospitality and Customer Service Management.', 'Respected builder and leader of customer-focused teams; strives to instill a shared, enthusiastic commitment to customer service.'], 'category': 'HR'}
{'id': '16852973', 'title': 'Highlights', 'content': ['Focused on customer satisfaction', 'Team management', 'Marketing savvy', 'Conflict resolution techniques', 'Training and development', 'Skilled multi-tasker', 'Client relations specialist'], 'category': 'HR'}
{'id': '16852973', 'title': 'Accomplishments', 'content': ['Missouri DOT Supervisor Training Certification', 'Certified by IHG in Customer Loyalty and Marketing by Segment', 'Hilton Worldwide General Manager Training Certification', 'Accomplished Trainer for cross server hospitality systems such as', 'Hilton OnQ', ',', 'Micros', 'Opera PMS', ', Fidelio', 'OPERA', 'Reservation System (ORS)', ',', 'Holidex', 'Complet